Download Data

In [6]:
!curl -X GET https://raw.githubusercontent.com/tensorflow/nmt/master/nmt/scripts/download_iwslt15.sh -o download_iwslt15.sh
!chmod +x download_iwslt15.sh && ./download_iwslt15.sh nmt_data

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1188  100  1188    0     0   9138      0 --:--:-- --:--:-- --:--:--  9138
mkdir: created directory 'nmt_data'
Download training dataset train.en and train.vi.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 12.9M  100 12.9M    0     0  10.3M      0  0:00:01  0:00:01 --:--:-- 10.3M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 17.2M  100 17.2M    0     0  13.8M      0  0:00:01  0:00:01 --:--:-- 13.7M
Download dev dataset tst2012.en and tst2012.vi.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Spee

In [2]:
!ls -l

total 8
drwxr-xr-x 2 root root 4096 Jan 17 12:04 nmt_model
drwxr-xr-x 1 root root 4096 Jan  8 17:15 sample_data


Clone code

In [3]:
!git clone https://github.com/tensorflow/nmt.git

Cloning into 'nmt'...
remote: Enumerating objects: 1247, done.
remote: Total 1247 (delta 0), reused 0 (delta 0), pack-reused 1247
Receiving objects: 100% (1247/1247), 1.22 MiB | 9.11 MiB/s, done.
Resolving deltas: 100% (892/892), done.


In [4]:
!ls -l nmt/nmt

total 236
-rw-r--r-- 1 root root  7387 Jan 17 12:05 attention_model.py
drwxr-xr-x 3 root root  4096 Jan 17 12:05 g3doc
-rw-r--r-- 1 root root 12252 Jan 17 12:05 gnmt_model.py
-rw-r--r-- 1 root root  8895 Jan 17 12:05 inference.py
-rw-r--r-- 1 root root  6490 Jan 17 12:05 inference_test.py
-rw-r--r-- 1 root root     0 Jan 17 12:05 __init__.py
-rw-r--r-- 1 root root 24395 Jan 17 12:05 model_helper.py
-rw-r--r-- 1 root root 33588 Jan 17 12:05 model.py
-rw-r--r-- 1 root root 48465 Jan 17 12:05 model_test.py
-rw-r--r-- 1 root root 29123 Jan 17 12:05 nmt.py
-rw-r--r-- 1 root root  3404 Jan 17 12:05 nmt_test.py
drwxr-xr-x 2 root root  4096 Jan 17 12:05 scripts
drwxr-xr-x 2 root root  4096 Jan 17 12:05 standard_hparams
drwxr-xr-x 2 root root  4096 Jan 17 12:05 testdata
-rw-r--r-- 1 root root 28948 Jan 17 12:05 train.py
drwxr-xr-x 2 root root  4096 Jan 17 12:05 utils


**Tensor Board**

Download

In [10]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!rm -rf ngrok
!unzip ngrok-stable-linux-amd64.zip

--2019-01-17 13:01:59--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 34.238.3.58, 34.231.75.48, 34.206.253.53, ...
Connecting to bin.equinox.io (bin.equinox.io)|34.238.3.58|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5363700 (5.1M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]   5.11M  20.4MB/s    in 0.3s    

2019-01-17 13:01:59 (20.4 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [5363700/5363700]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


Run

In [11]:
tsboard_cmd = 'tensorboard --logdir nmt_model --host 0.0.0.0 --port 6006 &'
print("Log dir", tsboard_cmd)
get_ipython().system_raw(tsboard_cmd)
get_ipython().system_raw('./ngrok http 6006 &')
!curl -s http://localhost:4040/api/tunnels | python3 -c "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

Log dir tensorboard --logdir nmt_model --host 0.0.0.0 --port 6006 &
http://6d65b2a9.ngrok.io


Training 

In [7]:
!mkdir nmt_model
!python -m nmt.nmt.nmt \
    --src=vi --tgt=en \
    --vocab_prefix=nmt_data/vocab  \
    --train_prefix=nmt_data/train \
    --dev_prefix=nmt_data/tst2012  \
    --test_prefix=nmt_data/tst2013 \
    --out_dir=nmt_model \
    --num_train_steps=12000 \
    --steps_per_stats=100 \
    --num_layers=4v \
    --num_units=128 \
    --dropout=0.2 \
    --metrics=bleu

mkdir: cannot create directory ‘nmt_model’: File exists
# Job id 0
2019-01-17 12:06:07.637538: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:964] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-01-17 12:06:07.638000: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1432] Found device 0 with properties: 
name: Tesla K80 major: 3 minor: 7 memoryClockRate(GHz): 0.8235
pciBusID: 0000:00:04.0
totalMemory: 11.17GiB freeMemory: 11.10GiB
2019-01-17 12:06:07.638039: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1511] Adding visible gpu devices: 0
2019-01-17 12:06:07.989907: I tensorflow/core/common_runtime/gpu/gpu_device.cc:982] Device interconnect StreamExecutor with strength 1 edge matrix:
2019-01-17 12:06:07.990000: I tensorflow/core/common_runtime/gpu/gpu_device.cc:988]      0 
2019-01-17 12:06:07.990048: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1001] 0:   N 
2019-01-17 12:06:0

In [9]:
!ls -l nmt_model

total 120612
drwxr-xr-x 3 root root     4096 Jan 17 12:49 best_bleu
-rw-r--r-- 1 root root      299 Jan 17 12:48 checkpoint
-rw-r--r-- 1 root root     2234 Jan 17 12:49 hparams
-rw-r--r-- 1 root root    12190 Jan 17 12:49 log_1547726771
-rw-r--r-- 1 root root   123967 Jan 17 12:49 output_dev
-rw-r--r-- 1 root root   121930 Jan 17 12:49 output_test
drwxr-xr-x 2 root root     4096 Jan 17 12:06 train_log
-rw-r--r-- 1 root root 23655940 Jan 17 12:41 translate.ckpt-10000.data-00000-of-00001
-rw-r--r-- 1 root root      712 Jan 17 12:41 translate.ckpt-10000.index
-rw-r--r-- 1 root root   982433 Jan 17 12:41 translate.ckpt-10000.meta
-rw-r--r-- 1 root root 23655940 Jan 17 12:45 translate.ckpt-11000.data-00000-of-00001
-rw-r--r-- 1 root root      712 Jan 17 12:45 translate.ckpt-11000.index
-rw-r--r-- 1 root root   982433 Jan 17 12:45 translate.ckpt-11000.meta
-rw-r--r-- 1 root root 23655940 Jan 17 12:48 translate.ckpt-12000.data-00000-of-00001
-rw-r--r-- 1 root root      712 Jan 17 12:48 transl

Attention

In [0]:
# Clean old model
!rm -rf nmt_attention_model
!mkdir nmt_attention_model

In [2]:
tsboard_cmd = 'tensorboard --logdir nmt_attention_model --host 0.0.0.0 --port 6006 &'
print("Log dir", tsboard_cmd)
get_ipython().system_raw(tsboard_cmd)
get_ipython().system_raw('./ngrok http 6006 &')
!curl -s http://localhost:4040/api/tunnels | python3 -c "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

Log dir tensorboard --logdir nmt_attention_model --host 0.0.0.0 --port 6006 &
http://e70637de.ngrok.io


In [1]:
!python -m nmt.nmt.nmt \
    --attention=scaled_luong \
    --src=vi --tgt=en \
    --vocab_prefix=nmt_data/vocab  \
    --train_prefix=nmt_data/train \
    --dev_prefix=nmt_data/tst2012  \
    --test_prefix=nmt_data/tst2013 \
    --out_dir=nmt_attention_model \
    --num_train_steps=12000 \
    --steps_per_stats=100 \
    --encoder_type=gnmt\
    --infer_mode=beam_search\
    --beam_width=10\
    --num_layers=4 \
    --num_units=128 \
    --dropout=0.2 \
    --metrics=bleu \
    --num_encoder_layers=8\
    --num_decoder_layers=8\
    --decay_scheme=luong10

mkdir: cannot create directory ‘nmt_attention_model’: File exists
# Job id 0
2019-01-17 13:30:53.654266: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:964] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-01-17 13:30:53.654691: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1432] Found device 0 with properties: 
name: Tesla K80 major: 3 minor: 7 memoryClockRate(GHz): 0.8235
pciBusID: 0000:00:04.0
totalMemory: 11.17GiB freeMemory: 11.10GiB
2019-01-17 13:30:53.654762: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1511] Adding visible gpu devices: 0
2019-01-17 13:30:54.008273: I tensorflow/core/common_runtime/gpu/gpu_device.cc:982] Device interconnect StreamExecutor with strength 1 edge matrix:
2019-01-17 13:30:54.008350: I tensorflow/core/common_runtime/gpu/gpu_device.cc:988]      0 
2019-01-17 13:30:54.008368: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1001] 0:   N 
2019-01-